# Training a simple CNN model in Tensorflow for Tornado Detection

This notebook steps through how to train a simple CNN model using a subset of TorNet.

This will not produce a model with any skill, but simply provides a working end-to-end example of how to set up a data loader, build, and fit a model


In [1]:
import sys
# Uncomment if tornet isn't installed in your environment or in your path already
#sys.path.append('../')  

import os
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tornet.data.tf.loader import create_tf_dataset 
from tornet.data.constants import ALL_VARIABLES

In [2]:
# Create basic dataloader
# This option loads directly from netcdf files, and will be slow and IO bound
# To speed up training, either
#     build as a tensorflow_dataset , (see tornet/data/tfds/tornet/README.md)
#     cache dataset first , or
#     use tf.data.Dataset.load on a pre-saved dataset

# Location of tornet
data_root = 'C:/Users/mjhig/tornet_2013'

# Get training data from 2018
data_type='train'
years = [2013,2014,2015,2016,2017,2018]

catalog_path = os.path.join(data_root,'catalog.csv')
if not os.path.exists(catalog_path):
    raise RuntimeError('Unable to find catalog.csv at '+data_root)
        
catalog = pd.read_csv(catalog_path,parse_dates=['start_time','end_time'])
catalog = catalog[catalog['type']==data_type]
catalog = catalog[catalog.start_time.dt.year.isin(years)]
catalog = catalog.sample(frac=1,random_state=1234)
file_list = [os.path.join(data_root,f) for f in catalog.filename]

ds = create_tf_dataset(file_list,variables=ALL_VARIABLES,n_frames=1) 

# (Optional) Save data for faster reloads (makes copy of data!)
#ds.save('tornet_sample.tfdataset') 


In [3]:
# If saved with ds.save(...), just load that model
#ds = tf.data.Dataset.load('tornet_sample.tfdataset')

In [4]:
# If data was registered in tensorflow_dataset, use that
# env variable TFDS_DATA_DIR should point to location of this resaved dataset
#import tensorflow_datasets as tfds
#import tornet.data.tfds.tornet.tornet_dataset_builder # registers 'tornet'

#data_type='train'
#years = [2018,]
#ds = tfds.load('tornet',split='+'.join(['%s-%d' % (data_type,y) for y in years]))

In [5]:
import tornet.data.preprocess as pp
from tornet.data import preprocess as tfpp

# Preprocess

# add 'coordinates' variable used by CoordConv layers
#ds = ds.map(lambda d: pp.add_coordinates(d,include_az=False,backend=tf))
     
# Take only last time frame
ds = ds.map(pp.remove_time_dim)

# Split sample into inputs,label
ds = ds.map(tfpp.split_x_y)

# (Optional) add sample weights
# weights={'wN':1.0,'w0':1.0,'w1':1.0,'w2':2.0,'wW':0.5}
# ds = ds.map(lambda x,y:  tfpp.compute_sample_weight(x,y,**weights) )

ds = ds.prefetch(tf.data.AUTOTUNE)
ds=ds.batch(32)

In [9]:
# Create a simple CNN model
# This normalizes data, concatenates along channel, and applies a Conv2D
import keras
from tornet.data.constants import CHANNEL_MIN_MAX
from tensorflow.keras.layers import (
    Conv2D, GlobalMaxPool2D,MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
)
input_vars = ALL_VARIABLES

# TF convention is B,L,W,H
inputs = {v:keras.Input(shape=(120,240,2),name=v) for v in input_vars}

# Normalize inputs
norm_layers = []
for v in input_vars:
    min_max = np.array(CHANNEL_MIN_MAX[v]) # [2,]

    # choose mean,var to get approximate [-1,1] scaling
    var=((min_max[1]-min_max[0])/2)**2 # scalar
    var=np.array(2*[var,])    # [n_sweeps,]
    offset=(min_max[0]+min_max[1])/2    # scalar
    offset=np.array(2*[offset,]) # [n_sweeps,]
    
    norm_layers.append(
        keras.layers.Normalization(mean=offset, variance=var,
                                   name='Normalized_%s' % v)
    )

# Concatenate normed inputs along channel dimension
x=keras.layers.Concatenate(axis=-1,name='Concatenate1')(
        [l(inputs[v]) for l,v in zip(norm_layers,input_vars)]
        )

# Replace background (nan) with -3
x=keras.layers.Lambda(lambda x: tf.where(tf.math.is_nan(x),-3.0,x),name='ReplaceNan')(x)

# **Improved Convolutional Feature Extraction**
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)  # Normalize activations
x = MaxPooling2D((2, 2))(x)  # Reduce spatial size
x = Dropout(0.3)(x)  # Regularization

x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.3)(x)


# **Final Prediction Layers**
x = GlobalAveragePooling2D()(x)  # Global feature aggregation
x = Dense(128, activation='relu')(x)  # Fully connected layer
x = Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)

y = Dense(1, activation='sigmoid', name='TornadoLikelihood')(x)

model = keras.Model(inputs=inputs,outputs=y,name='TornadoDetector')

model.summary()

Model: "TornadoDetector"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ DBZ (InputLayer)    │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ VEL (InputLayer)    │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ KDP (InputLayer)    │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ RHOHV (InputLayer)  │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ZDR (InputLayer)    │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ WIDTH (InputLayer)  │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalized_DBZ      │ (None, 120, 240,  │          0 │ DBZ[0][0]         │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalized_VEL      │ (None, 120, 240,  │          0 │ VEL[0][0]         │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalized_KDP      │ (None, 120, 240,  │          0 │ KDP[0][0]         │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalized_RHOHV    │ (None, 120, 240,  │          0 │ RHOHV[0][0]       │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalized_ZDR      │ (None, 120, 240,  │          0 │ ZDR[0][0]         │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalized_WIDTH    │ (None, 120, 240,  │          0 │ WIDTH[0][0]       │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Concatenate1        │ (None, 120, 240,  │          0 │ Normalized_DBZ[0… │
│ (Concatenate)       │ 12)               │            │ Normalized_VEL[0… │
│                     │                   │            │ Normalized_KDP[0… │
│                     │                   │            │ Normalized_RHOHV… │
│                     │                   │            │ Normalized_ZDR[0… │
│                     │                   │            │ Normalized_WIDTH… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ReplaceNan (Lambda) │ (None, 120, 240,  │          0 │ Concatenate1[0][… │
│                     │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 120, 240,  │      6,976 │ ReplaceNan[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 106,433 (415.75 KB)

 Trainable params: 106,049 (414.25 KB)

 Non-trainable params: 384 (1.50 KB)

In [7]:
# # Build a test set
# # Basic loader
# data_type='test'
# years = [2014]

# catalog_path = os.path.join(data_root,'catalog.csv')
# if not os.path.exists(catalog_path):
#     raise RuntimeError('Unable to find catalog.csv at '+data_root)
        
# catalog = pd.read_csv(catalog_path,parse_dates=['start_time','end_time'])
# catalog = catalog[catalog['type']==data_type]
# catalog = catalog[catalog.start_time.dt.year.isin(years)]
# catalog = catalog.sample(frac=1,random_state=1234)
# file_list = [os.path.join(data_root,f) for f in catalog.filename]

# ds_test = create_tf_dataset(file_list,variables=ALL_VARIABLES) 


# preprocess
# ds_test = ds_test.map(lambda d: pp.add_coordinates(d,include_az=False,backend=tf))
ds_test = ds_test.map(pp.remove_time_dim)
ds_test = ds_test.map(tfpp.split_x_y)
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)   

history_df = pd.DataFrame(history.history)
# Start the plot at epoch 5. You can change this to get a different view.
history_df.loc[:,['Precision','Recall','pr_auc']].plot()

In [17]:
import optuna
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, SGD,AdamW,Nadam,Adagrad
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.losses import BinaryCrossentropy,BinaryFocalCrossentropy
from tensorflow.keras.initializers import GlorotUniform

# Function to reset model weights
def reset_weights(model):
    """Reset model weights using GlorotUniform initialization."""
    for layer in model.layers:
        if hasattr(layer, 'kernel_initializer') and layer.weights:
            for weight in layer.weights:
                weight.assign(GlorotUniform()(weight.shape))

# Dice loss function
def dice_loss(y_true, y_pred, smooth=1e-6):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)
tf.random.set_seed(42)
# Define the objective function for Optuna
def objective(trial):
    # Sample hyperparameters
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    #optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'sgd','adamw','nadam','adagrad'])
    #loss_function = trial.suggest_categorical('loss_function', ['dice_loss', 'binary_crossentropy','binary_focal_crossentropy'])
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-3,log=True)
    #alpha = trial.suggest_float('alpha', 0.25, 0.50, step=0.05)    
    #gamma = trial.suggest_float('gamma', 0, 5,step=0.5 )
    #class_weights = {0: 1, 1: trial.suggest_float('pos_weight', 1.0, 10.0, step=0.5)}
    
    optimizer = AdamW(learning_rate=learning_rate, weight_decay=weight_decay)
    loss = BinaryFocalCrossentropy()
    # Select optimizer
    # if optimizer_name == 'adam':
    #     optimizer = Adam(learning_rate=learning_rate)
    # elif optimizer_name == 'sgd':
    #     optimizer = SGD(learning_rate=learning_rate, momentum=0.9)
    # elif optimizer_name == 'adamw':
    #     optimizer = AdamW(learning_rate=learning_rate, weight_decay=1e-4)
    # elif optimizer_name == 'nadam':
    #     optimizer = Nadam(learning_rate=learning_rate)
    # elif optimizer_name == 'adagrad':
    #     optimizer = Adagrad(learning_rate=learning_rate)

    # # Select loss function
    # if loss_function == 'dice_loss':
    #     loss = dice_loss
    # elif loss_function == 'binary_crossentropy':
    #     loss = BinaryCrossentropy()
    # elif loss_function == 'binary_focal_crossentropy':
    #     loss = BinaryFocalCrossentropy()
    
    


    # Reset model weights before training
    reset_weights(model)

    # Compile the model
    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=[
            AUC(curve='PR', name='pr_auc'),
            AUC(name='AUC'),
            Precision(name='Precision'),
            Recall(name='Recall')
        ]
    )

    # Early stopping
    early_stopping = EarlyStopping(
        monitor='pr_auc',
        patience=2,
        mode='max',
        restore_best_weights=True
    )

    # Train the model
    history = model.fit(
        ds,
        epochs=20,  # Limit epochs for tuning
        steps_per_epoch=10,
        callbacks=[early_stopping],
    )

    # Get the best PR AUC score
    best_pr_auc = max(history.history['pr_auc'])

    return best_pr_auc  # Optuna will maximize this

# Run Optuna optimization
#ds=ds.batch(32)
#study = optuna.create_study(direction='maximize')
#study.optimize(objective, n_trials=100)
model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=[
            AUC(curve='PR', name='pr_auc'),
            AUC(name='AUC'),
            Precision(name='Precision'),
            Recall(name='Recall')
        ]
    )

    # Early stopping
    early_stopping = EarlyStopping(
        monitor='pr_auc',
        patience=2,
        mode='max',
        restore_best_weights=True
    )




[I 2025-01-30 13:54:24,936] A new study created in memory with name: no-name-fdd19d78-33dd-4537-a5c9-402af605eeda


Epoch 1/20


c:\Users\mjhig\anaconda3\envs\tornet-torch\lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: {'DBZ': 'DBZ', 'VEL': 'VEL', 'KDP': 'KDP', 'RHOHV': 'RHOHV', 'ZDR': 'ZDR', 'WIDTH': 'WIDTH'}. Received: the structure of inputs={'DBZ': '*', 'VEL': '*', 'KDP': '*', 'RHOHV': '*', 'ZDR': '*', 'WIDTH': '*', 'range_folded_mask': '*', 'label': '*', 'category': '*', 'event_id': '*', 'ef_number': '*', 'az_lower': '*', 'az_upper': '*', 'rng_lower': '*', 'rng_upper': '*', 'time': '*', 'tornado_start_time': '*', 'tornado_end_time': '*'}
  warnings.warn(


10/10 ━━━━━━━━━━━━━━━━━━━━ 34s 3s/step - AUC: 0.6459 - Precision: 0.0495 - Recall: 0.7117 - loss: 0.2158 - pr_auc: 0.0609
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 27s 3s/step - AUC: 0.4557 - Precision: 0.1590 - Recall: 0.1691 - loss: 0.1346 - pr_auc: 0.1153
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 25s 3s/step - AUC: 0.5053 - Precision: 0.1423 - Recall: 0.1915 - loss: 0.0911 - pr_auc: 0.0927
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 25s 3s/step - AUC: 0.6214 - Precision: 0.0727 - Recall: 0.0130 - loss: 0.0760 - pr_auc: 0.0839     
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - AUC: 0.6182 - Precision: 0.2404 - Recall: 0.1761 - loss: 0.0914 - pr_auc: 0.1440
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 25s 3s/step - AUC: 0.6308 - Precision: 0.1590 - Recall: 0.0698 - loss: 0.0661 - pr_auc: 0.0804


[I 2025-01-30 13:57:05,833] Trial 0 finished with value: 0.14369924366474152 and parameters: {'learning_rate': 7.15907924760212e-05, 'weight_decay': 4.899194157588933e-05}. Best is trial 0 with value: 0.14369924366474152.


...

10/10 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - AUC: 0.3481 - Precision: 0.0319 - Recall: 0.5202 - loss: 0.3751 - pr_auc: 0.0448
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 28s 3s/step - AUC: 0.4927 - Precision: 0.0941 - Recall: 0.5728 - loss: 0.2668 - pr_auc: 0.1036
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 30s 3s/step - AUC: 0.5405 - Precision: 0.0464 - Recall: 0.3914 - loss: 0.2194 - pr_auc: 0.0619
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 29s 3s/step - AUC: 0.4455 - Precision: 0.0278 - Recall: 0.2166 - loss: 0.1805 - pr_auc: 0.0503


[I 2025-01-30 14:28:40,555] Trial 10 finished with value: 0.08534997701644897 and parameters: {'learning_rate': 2.266859176132713e-05, 'weight_decay': 0.0008815963139915432}. Best is trial 6 with value: 0.18140867352485657.


Epoch 1/20


c:\Users\mjhig\anaconda3\envs\tornet-torch\lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: {'DBZ': 'DBZ', 'VEL': 'VEL', 'KDP': 'KDP', 'RHOHV': 'RHOHV', 'ZDR': 'ZDR', 'WIDTH': 'WIDTH'}. Received: the structure of inputs={'DBZ': '*', 'VEL': '*', 'KDP': '*', 'RHOHV': '*', 'ZDR': '*', 'WIDTH': '*', 'range_folded_mask': '*', 'label': '*', 'category': '*', 'event_id': '*', 'ef_number': '*', 'az_lower': '*', 'az_upper': '*', 'rng_lower': '*', 'rng_upper': '*', 'time': '*', 'tornado_start_time': '*', 'tornado_end_time': '*'}
  warnings.warn(


10/10 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - AUC: 0.5160 - Precision: 0.1250 - Recall: 0.3502 - loss: 0.1761 - pr_auc: 0.0487
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 28s 3s/step - AUC: 0.5290 - Precision: 0.0000e+00 - Recall: 0.0000e+00 - loss: 0.1474 - pr_auc: 0.1343
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 28s 3s/step - AUC: 0.5321 - Precision: 0.0909 - Recall: 0.0083 - loss: 0.0730 - pr_auc: 0.1002     
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 38s 4s/step - AUC: 0.6117 - Precision: 0.1394 - Recall: 0.0714 - loss: 0.0825 - pr_auc: 0.0895
Epoch 5/20


In [38]:
# Evaluate
import tornet.metrics.keras.metrics as km
metrics = [keras.metrics.AUC(curve='PR',name='PRAUC'),
           km.BinaryAccuracy(from_logits=True,name='BinaryAccuracy'), 
           km.Precision(from_logits=True,name='Precision'),
           ]
model.compile(metrics=metrics)

# steps=10 for demo purposes
model.evaluate(ds_test,steps=10)


c:\Users\mjhig\anaconda3\envs\tornet-torch\lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: {'DBZ': 'DBZ', 'VEL': 'VEL', 'KDP': 'KDP', 'RHOHV': 'RHOHV', 'ZDR': 'ZDR', 'WIDTH': 'WIDTH'}. Received: the structure of inputs={'DBZ': '*', 'VEL': '*', 'KDP': '*', 'RHOHV': '*', 'ZDR': '*', 'WIDTH': '*', 'range_folded_mask': '*', 'label': '*', 'category': '*', 'event_id': '*', 'ef_number': '*', 'az_lower': '*', 'az_upper': '*', 'rng_lower': '*', 'rng_upper': '*', 'time': '*', 'tornado_start_time': '*', 'tornado_end_time': '*'}
  warnings.warn(


10/10 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - BinaryAccuracy: 0.1216 - PRAUC: 0.1009 - Precision: 0.1216 - loss: 0.8199


[0.8348420858383179,
 0.0908624678850174,
 0.10625000298023224,
 0.10625000298023224]